可参考的code：  
1.《Unsupervised Sentence Compression using Denoising Auto-Encoders》：https://github.com/zphang/usc_dae  
2.《Unsupervised Abstractive Sentence Summarization using Length Controlled Variational Autoencoder》：https://github.com/raphael-sch/SumVAE  
3.《SEQ^3: Differentiable Sequence-to-Sequence-to-Sequence Autoencoder for Unsupervised Abstractive Sentence Compression》：https://github.com/cbaziotis/seq3  
4.《Get To The Point: Summarization with Pointer-Generator Networks》：  
Tensorflow：https://github.com/abisee/pointer-generator  
PyTorch：https://github.com/atulkum/pointer_summarizer  
https://github.com/atulkum/pointer_summarizer  
5.pointer network：  
Tensorflow：https://github.com/devsisters/pointer-network-tensorflow  
PyTorch：https://github.com/shirgur/PointerNet  
5.Transformer：https://github.com/huggingface/transformers  
http://nlp.seas.harvard.edu/2018/04/03/attention.html  
https://github.com/tensorflow/tensor2tensor

**Modules**  
1.Embedding  
2.Network  
&emsp;2.1 Encoder1  
&emsp;2.2 Encoder2  
&emsp;2.3 Decoder  
&emsp;2.4 Discriminator  
&emsp;2.5 Loss calculation:  
&emsp;&emsp;2.5.1 loss1  
&emsp;&emsp;2.5.2 loss2  
&emsp;&emsp;2.5.3 loss3  
&emsp;&emsp;2.5.4 loss4  
3.Train

**Variants**  
1.Encoder1/Encoder2  
&emsp;1.1 RNN encoder  
&emsp;1.2 Transformer encoder  
&emsp;1.3 pointer network encoder  
2.Decoder  
3.Discriminator

**LIBS**

In [ ]:
import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F

**DATA & EMBEDDING**

In [ ]:
pass

**NETWORK**

In [ ]:
class EEDDModel(nn.Module):
  def __init__(Encoder1, Encoder2, Decoder, Discriminator, params):
    self.encoder1 = Encoder1(params['encoder1'])
    self.encoder2 = Encoder2(params['encoder2'])
    self.decoder = Decoder(params['decoder'])
    self.discriminator = Discriminator(params['discriminator'])
  
  def forward(self, embedded_inputs):
    abstract = self.encoder1(embedded_inputs)
    minor_info = self.encoder2(embedded_inputs)
    full_info = torch.cat(abstract, minor_info, dim = -1)
    recon1 = self.decoder(abstract)
    recon2 = self.decoder(minor_info)
    recon3 = self.decoder(full_info)
    return abstract, recon1, recon2, recon3


**LOSS**

In [ ]:
class EEDDLoss(nn.Module):
  def __init__(loss_ratios):
    self.ratios = loss_ratios
  
  def forward(self, origin, abstract, recon1, recon2, recon3, real_sentence):
    l1 = ReconLoss(origin, recon1)
    l2 = ReconLoss(origin, recon2)
    l3 = ReconLoss(origin, recon3)
    l4 = DiscriminatorLoss(real_sentence, abstract)
    loss = ratios[0]*l1 - ratios[1]*l2 + ratios[2]*l3 + ratios[3]*l4
    return loss

**TRAIN**

In [ ]:
def train(loss_ratios, model, Optimizer, opt_params, epochs, trainLoader, device):
  criterion = EEDDLoss(loss_ratios)
  optimizer = Optimizer(model.parameters(), lr=opt_params['lr'], momentum=opt_params['momentum'])   # eg: Optimizer = torch.optim.SGD
  model.to(device)
  for e in range(epochs):
    for i, embedded_inputs in enumerate(trainLoader):
      embedded_inputs = embedded_inputs.to(device)
      abstract, recon1, recon2, recon3 = model(embedded_inputs)
      loss = criterion(embedded_inputs, abstract, recon1, recon2, recon3, embedded_inputs)    # real_sentence is set as the original sentence for unsupervise
      optimizer.zero_grad()   # if don't call zero_grad, the grad of each batch will be accumulated
      loss.backward()
      optimizer.step()
      if i % 20 == 0:
        print('epoch: {}, batch: {}, loss: {}'.format(e+1, i+1, loss.data))
  torch.save(model, 'EEDD.pth')

In [ ]:
# back up

from modules.training.base_trainer import BaseTrainer

class EEDDTrainer(BaseTrainer):
  def __init__(model, train_loader, valid_loader, criterion, optimizers, config, device, batch_end_callbacks=None, loss_ratios):
    super().__init__(train_loader, valid_loader, config, device, batch_end_callbacks)
    self.model = model
    self.criterion = criterion
    self.optimizers = optimizers
  
  def train_epoch(self):
